In [13]:
# Import useful libs
import dask
import numpy as np
import xarray as xr
import dask.array as da
import scipy.fft as sp_fft
import scipy.signal as signal
from dask.distributed import Client, LocalCluster

In [14]:
# cluster = LocalCluster(n_workers=4)
client = Client()
client

c:\Users\baptiste.menetrier\.virtualenvs\phd-FBjz4hBd\Lib\site-packages\distributed\node.py:182: UserWarning: Port 8787 is already in use.
Perhaps you already have a cluster running?
Hosting the HTTP server on port 63529 instead
  warnings.warn(


Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: http://127.0.0.1:63529/status,
Dashboard: http://127.0.0.1:63529/status,Workers: 4
Total threads: 12,Total memory: 15.64 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:63532,Workers: 4
Dashboard: http://127.0.0.1:63529/status,Total threads: 12
Started: Just now,Total memory: 15.64 GiB
Comm: tcp://127.0.0.1:63551,Total threads: 3
Dashboard: http://127.0.0.1:63555/status,Memory: 3.91 GiB
Nanny: tcp://127.0.0.1:63535,


Load test case dataset 

In [15]:
fullpath = r"C:\Users\baptiste.menetrier\Desktop\devPy\phd\localisation\verlinden\verlinden_process_output\testcase1_0\ship\not_on_grid\hilbert_env_intercorr0\output_hilbert_env_intercorr0_noiseless.nc"
ds = xr.open_dataset(fullpath, chunks={'lon': 20, 'lat': 20})
ds = ds.isel(lon=slice(0, 50), lat=slice(0, 50))
ds

<xarray.Dataset> Size: 329MB
Dimensions:              (idx_rcv: 2, lat: 50, lon: 50, idx_rcv_pairs: 1,
                          idx_rcv_in_pair: 2, library_signal_time: 8192,
                          library_corr_lags: 16383, event_signal_time: 8192,
                          src_trajectory_time: 10, event_corr_lags: 16383)
Coordinates:
  * lon                  (lon) float64 400B -5.025 -5.023 ... -4.954 -4.952
  * lat                  (lat) float64 400B 52.49 52.49 52.49 ... 52.53 52.53
  * idx_rcv              (idx_rcv) int32 8B 0 1
  * idx_rcv_pairs        (idx_rcv_pairs) int32 4B 0
  * idx_rcv_in_pair      (idx_rcv_in_pair) int32 8B 0 1
  * library_signal_time  (library_signal_time) float32 33kB 0.0 0.0025 ... 20.48
  * library_corr_lags    (library_corr_lags) int32 66kB -8191 -8190 ... 8191
  * event_signal_time    (event_signal_time) float32 33kB 0.0 0.0025 ... 20.48
  * src_trajectory_time  (src_trajectory_time) float32 40B 0.0 9.719 ... 87.47
  * event_corr_lags      (event_corr_lags) int32 66kB -8191 -8190 ... 8190 8191
Data variables: (12/16)
    lon_rcv              (idx_rcv) float64 16B dask.array<chunksize=(2,), meta=np.ndarray>
    lat_rcv              (idx_rcv) float64 16B dask.array<chunksize=(2,), meta=np.ndarray>
    r_from_rcv           (idx_rcv, lat, lon) float64 40kB dask.array<chunksize=(2, 20, 20), meta=np.ndarray>
    delay_rcv            (idx_rcv, lat, lon) float64 40kB dask.array<chunksize=(2, 20, 20), meta=np.ndarray>
    rcv_pairs            (idx_rcv_pairs, idx_rcv_in_pair) int32 8B dask.array<chunksize=(1, 2), meta=np.ndarray>
    rcv_signal_library   (idx_rcv, lat, lon, library_signal_time) float32 164MB dask.array<chunksize=(2, 20, 20, 8192), meta=np.ndarray>
    ...                   ...
    az_src_rcv           (idx_rcv, src_trajectory_time) float32 80B dask.array<chunksize=(2, 10), meta=np.ndarray>
    rcv_signal_event     (idx_rcv, src_trajectory_time, event_signal_time) float32 655kB dask.array<chunksize=(2, 10, 8192), meta=np.ndarray>
    event_corr           (idx_rcv_pairs, src_trajectory_time, event_corr_lags) float32 655kB dask.array<chunksize=(1, 10, 16383), meta=np.ndarray>
    ambiguity_surface    (idx_rcv_pairs, src_trajectory_time, lat, lon) float64 200kB dask.array<chunksize=(1, 10, 20, 20), meta=np.ndarray>
    detected_pos_lon     (idx_rcv_pairs, src_trajectory_time) float64 80B dask.array<chunksize=(1, 10), meta=np.ndarray>
    detected_pos_lat     (idx_rcv_pairs, src_trajectory_time) float64 80B dask.array<chunksize=(1, 10), meta=np.ndarray>
Attributes:
    title:               Verlinden simulation with simple environment
    dx:                  100
    dy:                  100
    fullpath_populated:  C:\Users\baptiste.menetrier\Desktop\devPy\phd\locali...
    snr_dB:              Noiseless
    source_positions:    Not interpolated on grid
    src_pos:             not_on_grid
    long_name:           Ambiguity surface
    units:               dB
    detection_metric:    hilbert_env_intercorr0
    fullpath_output:     C:\Users\baptiste.menetrier\Desktop\devPy\phd\locali...
    fullpath_analysis:   C:\Users\baptiste.menetrier\Desktop\devPy\phd\img\lo...

In [16]:
def mult_along_axis(A, B, axis):
    # ensure we're working with Numpy arrays
    A = np.array(A)
    B = np.array(B)

    # shape check
    if axis >= A.ndim:
        raise np.AxisError(axis, A.ndim)
    if A.shape[axis] != B.size:
        raise ValueError(
            "Length of 'A' along the given axis must be the same as B.size"
        )

    # np.broadcast_to puts the new axis as the last axis, so
    # we swap the given axis with the last one, to determine the
    # corresponding array shape. np.swapaxes only returns a view
    # of the supplied array, so no data is copied unnecessarily.
    shape = np.swapaxes(A, A.ndim - 1, axis).shape

    # Broadcast to an array with the shape as above. Again,
    # no data is copied, we only get a new look at the existing data.
    B_brc = np.broadcast_to(B, shape)

    # Swap back the axes. As before, this only changes our "point of view".
    B_brc = np.swapaxes(B_brc, A.ndim - 1, axis)

    return A * B_brc

In [6]:
def derive_ambiguity(lib_data, event_data, src_traj_times, detection_metric):

    ambiguity_surface_dim = ["idx_rcv_pairs", "src_trajectory_time", "lat", "lon"]
    ambiguity_surface = np.empty(
        (1, len(src_traj_times)) + tuple(lib_data.sizes[d] for d in ["lat", "lon"])
    )

    da_amb_surf = xr.DataArray(
        data=ambiguity_surface,
        dims=ambiguity_surface_dim,
        coords={
            "idx_rcv_pairs": [lib_data.idx_rcv_pairs.values],
            "src_trajectory_time": src_traj_times,
            "lat": lib_data.lat.values,
            "lon": lib_data.lon.values,
        },
        name="ambiguity_surface",
    )

    for i_src_time, src_time in enumerate(src_traj_times):

        event_vector = event_data.sel(src_trajectory_time=src_time)

        if detection_metric == "intercorr0":
            amb_surf = mult_along_axis(
                lib_data,
                event_vector,
                axis=2,
            )
            autocorr_lib_0 = np.sum(lib_data.values**2, axis=2)
            autocorr_event_0 = np.sum(event_vector.values**2)
            # del lib_data, event_vector

            norm = np.sqrt(autocorr_lib_0 * autocorr_event_0)
            amb_surf = np.sum(amb_surf, axis=2) / norm  # Values in [-1, 1]
            amb_surf = (amb_surf + 1) / 2  # Values in [0, 1]
            da_amb_surf[dict(src_trajectory_time=i_src_time)] = amb_surf

        elif detection_metric == "lstsquares":
            lib = lib_data.values
            event = event_vector.values
            # del lib_data, event_vector

            diff = lib - event
            amb_surf = np.sum(diff**2, axis=2)  # Values in [0, max_diff**2]
            amb_surf = amb_surf / np.max(amb_surf)  # Values in [0, 1]
            amb_surf = (
                1 - amb_surf
            )  # Revert order so that diff = 0 correspond to maximum of ambiguity surface
            da_amb_surf[dict(src_trajectory_time=i_src_time)] = amb_surf

        elif detection_metric == "hilbert_env_intercorr0":
            lib_env = np.abs(signal.hilbert(lib_data))
            event_env = np.abs(signal.hilbert(event_vector))
            del lib_data, event_vector

            amb_surf = mult_along_axis(
                lib_env,
                event_env,
                axis=2,
            )

            autocorr_lib_0 = np.sum(lib_env**2, axis=2)
            autocorr_event_0 = np.sum(event_env**2)
            del lib_env, event_env

            norm = np.sqrt(autocorr_lib_0 * autocorr_event_0)
            amb_surf = np.sum(amb_surf, axis=2) / norm  # Values in [-1, 1]
            amb_surf = (amb_surf + 1) / 2  # Values in [0, 1]
            da_amb_surf[dict(src_trajectory_time=i_src_time)] = amb_surf

    return da_amb_surf

In [17]:
ambiguity_surface_dim = ["idx_rcv_pairs", "src_trajectory_time", "lat", "lon"]
ambiguity_surface = np.empty(tuple(ds.sizes[d] for d in ambiguity_surface_dim))

# Store all ambiguity surfaces in a list
ambiguity_surfaces = []

calc = []

for i_pair in ds.idx_rcv_pairs:
    lib_data = ds.library_corr.sel(idx_rcv_pairs=i_pair)
    event_data = ds.event_corr.sel(idx_rcv_pairs=i_pair)

    for i_src_time, src_time in enumerate(ds["src_trajectory_time"]):

        event_vector = event_data.sel(src_trajectory_time=src_time)

        amb_surf = mult_along_axis(
            lib_data,
            event_vector,
            axis=2,
        )
        autocorr_lib_0 = np.sum(lib_data.values**2, axis=2)
        autocorr_event_0 = np.sum(event_vector.values**2)
        # del lib_data, event_vector

        norm = np.sqrt(autocorr_lib_0 * autocorr_event_0)
        amb_surf = np.sum(amb_surf, axis=2) / norm  # Values in [-1, 1]
        amb_surf = (amb_surf + 1) / 2  # Values in [0, 1]

        calc.append(amb_surf)



    # # Init pool
    # pool = multiprocessing.Pool(processes=N_CORES)
    # # Build the parameter pool
    # idx_ship_intervalls = np.array_split(
    #     np.arange(ds.dims["src_trajectory_time"]), N_CORES
    # )
    # src_traj_time_intervalls = np.array_split(ds["src_trajectory_time"], N_CORES)
    # # param_pool = [
    # #     (ds, i_pair, idx_ship_intervalls[i], detection_metric)
    # #     for i in range(len(idx_ship_intervalls))
    # # ]
    # param_pool = [
    #     (lib_data, event_data, src_traj_time_intervalls[i], detection_metric)
    #     for i in range(len(idx_ship_intervalls))
    # ]
    # # Run parallel processes
    # results = pool.starmap(derive_ambiguity, param_pool)
    # # Close pool
    # pool.close()
    # # Wait for all processes to finish
    # pool.join()

KeyboardInterrupt: 

In [8]:
lib_data

<xarray.DataArray 'library_corr' (lat: 171, lon: 171, library_corr_lags: 16383)> Size: 2GB
dask.array<getitem, shape=(171, 171, 16383), dtype=float32, chunksize=(20, 20, 16383), chunktype=numpy.ndarray>
Coordinates:
  * lon                (lon) float64 1kB -5.025 -5.023 -5.022 ... -4.775 -4.774
  * lat                (lat) float64 1kB 52.49 52.49 52.49 ... 52.64 52.64 52.64
    idx_rcv_pairs      int32 4B 0
  * library_corr_lags  (library_corr_lags) int32 66kB -8191 -8190 ... 8190 8191
Attributes:
    long_name:  $R_{ij}^{l}(\tau)$